In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import binom

In [3]:
DECAY = 0.8  # exponential MA decay

In [4]:
def beta_mean(a, b):
    return a / (a + b)

def beta_sd(a, b):
    return np.sqrt(a * b / (((a + b) ** 2) * (a + b + 1)))

In [5]:
def price(ledger):
    results = []
    for c, q in ledger[::-1]:
        results.extend(q * [c / q])
    weights = [DECAY ** i for i in range(len(results))]
    px = sum([w * r for w, r in zip(weights, results)]) / sum(weights)
    return round(px * 100.0, 1)

In [20]:
class QuizBeta():
    def __init__(self):
        self.ledger = []
        
        # Beta distribution params
        self.a = 3
        self.b = 3
    
    def __repr__(self):
        return str(self.__dict__)
    
    def round_update(self, correct, questions):
        self.ledger.append((correct, questions))
        self.beta_update(correct, questions)
    
    def beta_update(self, correct, questions):
        self.a += correct
        self.b += questions - correct
    
    def price(self):
        if len(self.ledger):
            return price(self.ledger)
        return 50
    
    def price_future(self, questions_left):
        if questions_left <= 0:
            return price(self.ledger), price(self.ledger)
        k = 0.01  # the spread is (mean - k * questions_left * sigma, mean + k * sigma)
        
        lower_p = max(0.0, beta_mean(self.a, self.b) - k * questions_left * beta_sd(self.a, self.b))
        upper_p = min(1.0, beta_mean(self.a, self.b) + k * questions_left * beta_sd(self.a, self.b))

        lower_ledger = self.ledger + [(lower_p * questions_left, questions_left)]
        upper_ledger = self.ledger + [(upper_p * questions_left, questions_left)]
                
        return price(lower_ledger), price(upper_ledger)

In [38]:
class Future():
    def __init__(self, player, tenor, name):
        self.name = name
        self.player = player.beta
        self.tenor = tenor
        self.expired = False
    
    def __repr__(self):
        return str(self.__dict__)
        
    def buy_price(self):
        return self.player.price_future(self.tenor)[1]
    
    def sell_price(self):
        return self.player.price_future(self.tenor)[0]
    
    def roll(self, questions):
        self.tenor -= questions
        self.tenor = max(0, self.tenor)
        if self.tenor == 0:
            self.expired = True

In [39]:
class Spot():
    def __init__(self, player, name):
        self.name = name
        self.player = player.beta
        self.expired = False
    
    def __repr__(self):
        return str(self.__dict__)
        
    def buy_price(self):
        return self.player.price()
    
    def sell_price(self):
        return self.player.price()
    
    def roll(self, questions):
        pass

In [40]:
class Portfolio():
    def __init__(self):
        self.cash = 100.0
        self.contracts = {}
        
    def value(self):
        total = self.cash
        for contract, volume in self.contracts.values():
            if volume < 0:
                total -= volume * contract.buy_price()
            else:
                total += volume * contract.sell_price()
        return total
    
    def roll(self, questions):
        for contract, volume in self.contracts.values():
            contract.roll(questions)
            if contract.expired:
                if volume < 0:
                    self.buy(contract, volume)
                else:
                    self.sell(contract, volume)
                
    
    def buy(self, contract, volume):
        if contract.name in self.contracts.keys():
            current_volume = self.contracts[contract.name][1]
            self.contracts[contract.name] = (contract, current_volume + volume)
        else:
            self.contracts[contract.name] = (contract, volume)
        self.cash -= contract.buy_price() * volume
        
    def sell(self, contract, volume):
        if contract.name in self.contracts.keys():
            current_volume = self.contracts[contract.name][1]
            self.contracts[contract.name] = (contract, current_volume - volume)
        else:
            self.contracts[contract.name] = (contract, -volume)
        self.cash += contract.sell_price() * volume
        
    def __repr__(self):
        return '\n'.join([f'"{name}" x{volume}' for name, (_, volume) in self.contracts.items()]
                         + [f'CASH: {self.cash}'])

In [41]:
class Player():
    def __init__(self, name):
        self.name = name
        self.beta = QuizBeta()
        self.portfolio = Portfolio()
    
    def __repr__(self):
        return str(self.__dict__)

In [42]:
class Quiz():
    def __init__(self, players):
        self.players = players
        
    def round_update(self, scores, total):
        if set(scores.keys()) != set(self.players.keys()):
            raise ValueError('Missing some players or too many players')
        for player, score in scores.items():
            self.players[player].beta.round_update(score, total)
            self.players[player].portfolio.roll(total)


In [43]:
def player_values(quiz):
    for name, player in quiz.players.items():
        print(f'{name}: Q${player.portfolio.value():.1f}')

In [44]:
ben = Player('Ben')
tom = Player('Tom')
jerry = Player('Jerry')

In [45]:
quiz = Quiz({'Ben': ben, 'Tom': tom, 'Jerry': jerry})

In [46]:
player_values(quiz)

Ben: Q$100.0
Tom: Q$100.0
Jerry: Q$100.0


In [47]:
ben.portfolio.buy(Spot(tom, 'tom'), 1)

In [48]:
player_values(quiz)

Ben: Q$100.0
Tom: Q$100.0
Jerry: Q$100.0


In [51]:
quiz.round_update({'Ben': 3, 'Jerry': 10, 'Tom': 10}, 10)

In [52]:
player_values(quiz)

Ben: Q$145.2
Tom: Q$100.0
Jerry: Q$100.0


In [53]:
quiz.round_update({'Ben': 3, 'Tom': 10, 'Jerry': 5}, 10)

In [54]:
player_values(quiz)

Ben: Q$149.5
Tom: Q$100.0
Jerry: Q$100.0


In [55]:
quiz.round_update({'Ben': 3, 'Tom': 0, 'Jerry': 5}, 10)

In [56]:
player_values(quiz)

Ben: Q$60.7
Tom: Q$100.0
Jerry: Q$100.0
